In [1]:
from dotenv import load_dotenv
import os
import pandas as pd

import time
import json

import logging

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options

from selenium.common.exceptions import (NoSuchElementException, NoSuchFrameException, NoSuchWindowException, 
                                        NoAlertPresentException, ElementNotVisibleException, ElementNotSelectableException, 
                                        StaleElementReferenceException, TimeoutException, WebDriverException, 
                                        MoveTargetOutOfBoundsException, InvalidElementStateException, 
                                        UnexpectedAlertPresentException, JavascriptException, InvalidSelectorException)


# Cargar las variables de entorno
load_dotenv()
search_artist = os.getenv("search_artist")
path = os.getenv("path_driver")
logging.basicConfig(level=logging.INFO)

Cargar primera pagina con la que se obtubieron la lista de artista.

In [2]:
service = Service(executable_path=path)
options = Options()
#options.headless = True  # Ejecutar en modo sin cabeza (headless)
options.add_argument("--headless")
options.add_argument('--disable-gpu')  # Desactivar GPU, necesario para algunos sistemas
options.add_argument("--disable-images")
options.add_argument('--no-sandbox')  # Añadir sandbox (opcional, puede ser necesario en algunos entornos)
options.add_argument('--disable-dev-shm-usage')  # Solucionar problemas de recursos compartidos en contenedores
options.add_argument('start-maximized')  # Iniciar maximizado
options.add_argument('window-size=1920x1080')  # Tamaño de ventana (resolución)


In [31]:
# Cargar el archivo JSON
with open('Artista_limpio.json', 'r', encoding='utf-8') as file:
    dic = json.load(file)
prox_concerts = []


# Inicializar el driver
driver = webdriver.Chrome(service=service, options=options)
# Abrir la URL
driver.get(search_artist)


try:
    # Iterar sobre cada enlace en dic['link'] 
    prox_concerts = []
    for indice, artistas_link in enumerate(dic['link']):
        # indice+=617
        driver.get(artistas_link)
        page_source = driver.page_source
        time.sleep(10)
        # load_more_buttons = WebDriverWait(driver, 30).until(
        #     EC.visibility_of_element_located((By.XPATH, "//div[@class = 'OsDXzJEUtjxbMlCnPMsj']"))
        # )

        load_more_buttons = driver.find_elements(By.XPATH, "//div[@class = 'OsDXzJEUtjxbMlCnPMsj']")
        fechas_web = driver.find_elements(By.XPATH, '//div[@class = "kRBUyvXPIcHrLiP1xayw"]')
        veneu_web = driver.find_elements(By.XPATH, '//div[@class = "TYzA8d85IfvLeyChcYJj"]')
        site_web = driver.find_elements(By.XPATH, '//div[@class = "D9Nc3q2GrC4mEVUaPKoR"]')

        
        if load_more_buttons:
            for i, button in enumerate(load_more_buttons):
                if "Show More Dates" in button.text:
                    if i == 1:
                        driver.execute_script("arguments[0].scrollIntoView();", button)
                        button.click()
                    elif i == 0 and len(load_more_buttons) == 1:
                        driver.execute_script("arguments[0].scrollIntoView();", button)
                        button.click()
        else:
            pass



        if fechas_web and veneu_web and site_web:

            fechas = [div.text.replace("\n", " ") for div in fechas_web]
            fechas_limpias = [fecha for fecha in fechas if fecha.strip()]

            veneu = [div.text.replace("\n", " ") for div in veneu_web]
            veneu_limpias = [veneu for veneu in veneu if veneu.strip()]

            site = [div.text.replace("\n", " ") for div in site_web]
            site_limpias = [site for site in site if site.strip()]

            new_dic = {'fecha':fechas_limpias,'Veneu':veneu_limpias,'Ciudad':site_limpias  }
            print(f"{indice}-{dic['Artista'][indice]}: {new_dic}")
            #print(f"new_dic de {artistas_link}")
            #print(new_dic)
            prox_concerts.append(new_dic)

        else:
            new_dic = {'fecha':None,'Veneu':None,'Ciudad':None  }
            print(f"{indice} - {dic['Artista'][indice]} no tiene fechas de evento")
            prox_concerts.append(new_dic)

    dic['prox_concerts'] = prox_concerts
    
    with open('DB.json', 'w', encoding='utf-8') as file:
        json.dump(dic, file, indent=4)
        
except (NoSuchElementException, NoSuchFrameException, NoSuchWindowException, 
        NoAlertPresentException, ElementNotVisibleException, ElementNotSelectableException, 
        StaleElementReferenceException, TimeoutException, WebDriverException, 
        MoveTargetOutOfBoundsException, InvalidElementStateException, 
        UnexpectedAlertPresentException, JavascriptException, InvalidSelectorException) as e:
    
    print(f"Error en el índice {indice}: {str(e)}")
    
    with open('DB.json', 'w', encoding='utf-8') as file:
        json.dump(dic, file, indent=4,ensure_ascii=False)
    driver.quit()

        
finally:
    # Cierra el navegador
    driver.quit()

0-Taylor Swift: {'fecha': ['AUG 15', 'AUG 16', 'AUG 17', 'AUG 19', 'AUG 20', 'OCT 18'], 'Veneu': ['Wembley Stadium', 'Wembley Stadium', 'Wembley Stadium', 'Wembley Stadium', 'Wembley Stadium', 'Hard Rock Stadium'], 'Ciudad': ['London, United Kingdom', 'London, United Kingdom', 'London, United Kingdom', 'London, United Kingdom', 'London, United Kingdom', 'Miami Gardens, FL']}
1 - Katy Perry no tiene fechas de evento
2-Bruno Mars: {'fecha': ['AUG 10', 'AUG 15', 'AUG 16', 'AUG 20', 'AUG 23', 'AUG 24'], 'Veneu': ['Estadio GNP Seguros', 'Intuit Dome', 'Intuit Dome', 'Dolby Live', 'Dolby Live', 'Dolby Live'], 'Ciudad': ['Ciudad Universitaria, Mexico', 'Inglewood, CA', 'Inglewood, CA', 'Las Vegas, NV', 'Las Vegas, NV', 'Las Vegas, NV']}
3-Maroon 5: {'fecha': ['SEP 27', 'SEP 28', 'OCT 02', 'OCT 04', 'OCT 05', 'OCT 09'], 'Veneu': ['Dolby Live', 'Dolby Live', 'Dolby Live', 'Dolby Live', 'Dolby Live', 'Dolby Live'], 'Ciudad': ['Las Vegas, NV', 'Las Vegas, NV', 'Las Vegas, NV', 'Las Vegas, NV', 'L

In [3]:
# Cargar el archivo JSON
with open('Artista.json', 'r', encoding='utf-8') as file:
    data = json.load(file)

# Convertir a DataFrame

df = pd.DataFrame(data)

df_sin_duplicados = df.drop_duplicates()

dic = df_sin_duplicados.to_dict(orient='list')

# with open('Artista_limpio.json', 'w', encoding='utf-8') as file:
#         json.dump(dic, file, indent=4)

In [6]:
new_dic = {'fecha':None,'Veneu':None,'Ciudad':None  }
print(f"{indice} - {dic['Artista'][indice]} no tiene fechas de evento")
prox_concerts.append(new_dic)

617 - Weezer no tiene fechas de evento


In [9]:
print(prox_concerts)
dic['prox_concerts'] = prox_concerts

[{'fecha': ['AUG 15', 'AUG 16', 'AUG 17', 'AUG 19', 'AUG 20', 'OCT 18'], 'Veneu': ['Wembley Stadium', 'Wembley Stadium', 'Wembley Stadium', 'Wembley Stadium', 'Wembley Stadium', 'Hard Rock Stadium'], 'Ciudad': ['London, United Kingdom', 'London, United Kingdom', 'London, United Kingdom', 'London, United Kingdom', 'London, United Kingdom', 'Miami Gardens, FL']}, {'fecha': None, 'Veneu': None, 'Ciudad': None}, {'fecha': ['AUG 10', 'AUG 15', 'AUG 16', 'AUG 20', 'AUG 23', 'AUG 24'], 'Veneu': ['Estadio GNP Seguros', 'Intuit Dome', 'Intuit Dome', 'Dolby Live', 'Dolby Live', 'Dolby Live'], 'Ciudad': ['Ciudad Universitaria, Mexico', 'Inglewood, CA', 'Inglewood, CA', 'Las Vegas, NV', 'Las Vegas, NV', 'Las Vegas, NV']}, {'fecha': ['SEP 27', 'SEP 28', 'OCT 02', 'OCT 04', 'OCT 05', 'OCT 09'], 'Veneu': ['Dolby Live', 'Dolby Live', 'Dolby Live', 'Dolby Live', 'Dolby Live', 'Dolby Live'], 'Ciudad': ['Las Vegas, NV', 'Las Vegas, NV', 'Las Vegas, NV', 'Las Vegas, NV', 'Las Vegas, NV', 'Las Vegas, NV']}

In [13]:
with open('DB.json', 'w', encoding='utf-8') as file:
    json.dump(dic, file, indent=4,ensure_ascii=False)

In [ ]:
dic['Artista'], dic['link'] = extraer_artistas(driver, inicio=ultimo_indice)

# Guardar los datos actualizados en el archivo JSON
with open('DB_actualizado.json', 'w', encoding='utf-8') as file:
    json.dump(dic, file, ensure_ascii=False, indent=4)

In [34]:
driver = webdriver.Chrome(service=service, options=options)
driver.get('https://www.bandsintown.com/u/explore?')

load_more = driver.find_element(By.XPATH, "//div[@class = 'aM5QHbhlJjUYWcPVV0Am']")
num = 0      


while load_more.text=="Show More":
        
        num+=1
        
        #print(i)
        load_more.click()
        time.sleep(5)

        print(f'numero de clicks:{num}')

        # div_elements = WebDriverWait(driver, 10).until(
        #     EC.presence_of_all_elements_located((By.XPATH, '//div[@class = "y780Kgtic5r7Cod_zQVq"]'))
        # )
        # div_texts = [div.text.replace("\nView Concerts", "") for div in div_elements]

        # a_elements = WebDriverWait(driver, 10).until(
        #     EC.presence_of_all_elements_located((By.XPATH, '//a[@class = "tlhOE0qLeWoH8o_cp5ZE"]'))
        # )
        # hrefs = [a.get_attribute('href') for a in a_elements]
        
        

        # print(f'Num de artistas: {len(div_texts)}')

        # load_more_buttons = driver.find_elements(By.XPATH, "//div[@class = 'aM5QHbhlJjUYWcPVV0Am']")

        # # Verificar si hemos alcanzado el límite
        # if len(div_texts) > 601:
        #     break

numero de clicks:1
numero de clicks:2
numero de clicks:3
numero de clicks:4
numero de clicks:5
